# **Aprendizaje supervisado**
# SL10. Random forest

Echaremos un vistazo a la motivación de un poderoso algoritmo: un algoritmo no paramétrico llamado **Random Forest**.
Random Forest es un ejemplo de un método de **ensamblado** , lo que significa que se basa en la agregación de los resultados de un conjunto de estimadores más simples.
El resultado algo sorprendente con tales métodos de conjunto es que la suma puede ser mayor que las partes: es decir, ¡un voto mayoritario entre varios estimadores puede terminar siendo mejor que cualquiera de los estimadores individuales que votan!
Veremos ejemplos de esto en las siguientes secciones.
Comenzamos con las importaciones estándar:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

## <font color='blue'>**Random Forest**</font>

Random forest es un ejemplo de un **ensamblado** construido sobre árboles de decisión.
Por esta razón, comenzaremos discutiendo los propios árboles de decisión.

1. Los árboles de decisión a menudo imitan el pensamiento a nivel humano, por lo que es muy sencillo comprender los datos y hacer algunas buenas interpretaciones.

2. Los árboles de decisión le hacen ver la lógica de los datos a interpretar (no como los algoritmos de caja negra como SVM, ANN, etc.).

3. Un árbol de decisión es un árbol donde cada nodo representa una característica (atributo), cada vínculo (rama) representa una decisión (regla) y cada hoja representa un resultado (valor categórico o continuo).

Los árboles de decisión son formas extremadamente intuitivas de clasificar o etiquetar objetos: simplemente haga una serie de preguntas diseñadas para concentrarse en la clasificación.
Por ejemplo, si quisiera construir un árbol de decisiones para clasificar un animal con el que se encuentra durante una caminata, puede construir el que se muestra aquí:

![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/05.08-decision-tree.png?raw=1)


La división binaria hace que esto sea extremadamente eficiente: en un árbol bien construido, cada pregunta reducirá el número de opciones a aproximadamente la mitad, reduciendo muy rápidamente las opciones incluso entre un gran número de clases.
El truco, por supuesto, está en decidir qué preguntas hacer en cada paso.
En las implementaciones de aprendizaje automático de árboles de decisión, las preguntas generalmente toman la forma de divisiones alineadas con el eje en los datos: es decir, cada nodo del árbol divide los datos en dos grupos utilizando un valor de corte dentro de una de las características.
Veamos ahora un ejemplo de esto.

## Creando Árboles de Decisión

Considere los siguientes datos bidimensionales, que tienen una de cuatro etiquetas de clase:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

Un árbol de decisión simple construido sobre estos datos dividirá iterativamente los datos a lo largo de uno u otro eje de acuerdo con algún criterio cuantitativo, y en cada nivel asignará la etiqueta de la nueva región de acuerdo con un voto mayoritario de puntos dentro de ella.
Esta figura presenta una visualización de los primeros cuatro niveles de un clasificador de árbol de decisión para estos datos:

![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/05.08-decision-tree-levels.png?raw=1)


Observe que después de la primera división, todos los puntos de la rama superior permanecen sin cambios, por lo que no es necesario subdividir más esta rama.
A excepción de los nodos que contienen todos de un color, en cada nivel * cada * región se divide nuevamente a lo largo de una de las dos características.

Este proceso de ajustar un árbol de decisiones a nuestros datos se puede realizar en Scikit-Learn con el estimador `` DecisionTreeClassifier '':

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)

Escribamos una función de utilidad rápida para ayudarnos a visualizar la salida del clasificador:

In [ ]:
a = np.array([[1, 2, 3]])
b = np.array([[10, 20, 30, 40]])
c = np.array([[1, 2, 3],
             [4, 5, 6]
             ])

In [ ]:
a

In [ ]:
a.shape

In [ ]:
x1, y1 = np.meshgrid(a, b)
print(x1)
print(y1)

In [ ]:
c.ravel()

In [ ]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()

    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap, clim=(y.min(), y.max()),
                           zorder=1)

    ax.set(xlim=xlim, ylim=ylim)

Ahora podemos examinar cómo se ve la clasificación del árbol de decisión:

In [ ]:
visualize_classifier(DecisionTreeClassifier(), X, y)

Observe que a medida que aumenta la profundidad, tendemos a obtener regiones de clasificación de formas muy extrañas; por ejemplo, a una profundidad de cinco, hay una región púrpura alta y delgada entre las regiones amarilla y azul.
Está claro que esto es menos un resultado de la verdadera distribución intrínseca de los datos y más un resultado de las propiedades particulares de muestreo o ruido de los datos.
Es decir, este árbol de decisiones, incluso a solo cinco niveles de profundidad, claramente sobrepasa nuestros datos.

## Árboles de Decisión y Overfitting

Tal sobreajuste resulta ser una propiedad general de los árboles de decisión: es muy fácil profundizar demasiado en el árbol y, por lo tanto, ajustar los detalles de los datos particulares en lugar de las propiedades generales de las distribuciones de las que se extraen.
Otra forma de ver este ajuste excesivo es observar modelos entrenados en diferentes subconjuntos de datos; por ejemplo, en esta figura entrenamos dos árboles diferentes, cada uno en la mitad de los datos originales:

![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/05.08-decision-tree-overfitting.png?raw=1)

Está claro que en algunos lugares, los dos árboles producen resultados consistentes (por ejemplo, en las cuatro esquinas), mientras que en otros lugares, los dos árboles dan clasificaciones muy diferentes (por ejemplo, en las regiones entre dos grupos).
La observación clave es que las inconsistencias tienden a ocurrir donde la clasificación es menos segura y, por lo tanto, al usar información de * ambos * de estos árboles, ¡podríamos obtener un mejor resultado!

Si está ejecutando este cuaderno en vivo, la siguiente función le permitirá mostrar interactivamente los ajustes de árboles entrenados en un subconjunto aleatorio de los datos:

Así como el uso de información de dos árboles mejora nuestros resultados, podríamos esperar que el uso de información de muchos árboles mejore nuestros resultados aún más.

## Algoritmos

1. CART (árboles de clasificación y regresión) → utiliza el índice de Gini (clasificación) como métrica.

2. ID3 (dicotomizador iterativo 3) → utiliza la función de entropía y la ganancia de información como métricas.

**Entropía:** La entropía de un conjunto de datos, es una medida de la impureza o desorden, del conjunto de datos La entropía también puede pensarse, como una medida de incertidumbre. Deberíamos intentar minimizar la entropía. El objetivo de los modelos de aprendizaje automático es reducir la incertidumbre o la entropía, en la medida de lo posible.

$$ H(S) = \sum_{s\in S}-p(s)log_2 p(s)$$
Donde s son las clases de S.

**Ganancia de información:** La ganancia de información es una medida de cuánta información nos brinda una característica sobre las clases. El algoritmo de árboles de decisión siempre intentará maximizar la ganancia de información. Característica, que divide perfectamente los datos, debe dar la máxima información. Una función, con la mayor ganancia de información, se utilizará para dividir primero.

$$ IG(S,a) = H(S) - H(S|a)$$
Como calculamos  $IG(S,a)$. Supongamos un conjunto donde 13 valores son 0 y 7 valores son 1. Luego, tenemos una variable $a$ que tiene dos valores valor0 y valor 1. Posteriormente separamos por esta variable obteniendo los siguientes conjuntos. 8(7|1) y 12(6|6). ¿Cuanto vale IG(S,a)?

$$ IG(S,a) = H(\frac{13}{20}, \frac{7}{20}) - (H(\frac{7}{8}, \frac{1}{8}) + H(\frac{6}{12}, \frac{6}{12}))$$

**Indice de Gini:** El índice de Gini, también conocido como impureza de Gini, es una medida de entropía y calcula  de probabilidad de un elemento que pertenece a una categoria. se clasifica incorrectamente cuando se selecciona al azar. Si el elemento pertenece a la categoria $i$ la probabilidad de ser mal clasificado esta dado por $ \sum_{i \neq k} p_k = 1 - pi$

 Gini Index (GI) = $$ \sum_{i \in S} p_i \sum_{j \neq i} p_j = \sum_{i \in S} p_i(1-p_i) = \sum_{i \in S} p_i - \sum_{i \in S} p_i^2 =  1 - \sum_{i \in S} (P_i)^2$$, por ejemplo supongamos que tenemos 4 bolas rojas y 0 azul. EL indice de Gini es:

$$ I_G = 1 - (1^2 + 0^2)=1$$
Caso 2 y 2 tenemos:
$$ I_G = 1 - ((\frac{1}{2})^2 + (\frac{1}{2})^2)=0.5$$

Caso 3 rojas una azul
$$ I_G = 1 - ((\frac{3}{4})^2 + (\frac{1}{4})^2)=0.375$$

Al igual que con la entropía para estimar la mejor separación, se utiliza la ganancia del coeficiente de Gini.

## <font color='green'>Actividad 1</font>

Construya un arbol de decisión con los datos proporcionados e interprete este. Apóyese en los códigos siguientes.



```
from sklearn.tree import DecisionTreeClassifier
```

Considere utilizar variables dummy para poder realizar la separación.



```
one_hot_data = pd.get_dummies
```

Y utilice el siguiente código para visualizar el resultado del arbol.



```
# Export/Print un arbol de decisión en formato .dot.
print(tree.export_graphviz(clf_train, None))

# Crea la data.
dot_data = tree.export_graphviz(clf_train, out_file=None, feature_names=list(one_hot_data.columns.values),
                                class_names=['Not_Play', 'Play'], rounded=True, filled=True) #Gini o Entropia.
#Creaamos el grafico.
graph = pydotplus.graph_from_dot_data(dot_data)

# Mostramos el gráfico.
Image(graph.create_png())
```



In [ ]:
## importemos dependencias
from sklearn import tree
import pandas as pd
import pydotplus
from IPython.display import Image


In [ ]:
# Creemos el dataset

golf_df = pd.DataFrame()

#agreguemos outlook
golf_df['Outlook'] = ['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy',
                     'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast',
                     'overcast', 'rainy']

#agreguemos temperature
golf_df['Temperature'] = ['hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool',
                         'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild']

#agreguemos humidity
golf_df['Humidity'] = ['high', 'high', 'high', 'high', 'normal', 'normal', 'normal',
                      'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high']

#agreguemos windy
golf_df['Windy'] = ['false', 'true', 'false', 'false', 'false', 'true', 'true',
                   'false', 'false', 'false', 'true', 'true', 'false', 'true']

#Se juega o no play?
golf_df['Play'] = ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes',
                  'yes', 'yes', 'no']


#Print/show the new data
print(golf_df)

In [ ]:
# Tú codigo aquí

<font color='green'>Fin Actividad 1</font>

## <font color='blue'>**Ensamblado de Estimadores: Random Forest**</font>

Ensemble es un concepto de Machine Learning en el que la idea es entrenar múltiples modelos utilizando el mismo algoritmo de aprendizaje. Los conjuntos participan en un grupo más grande de métodos, llamados multiclasificadores, donde un conjunto de cientos o miles de modelos con un objetivo común se fusionan para resolver el problema.

Las principales causas de error en el aprendizaje se deben al ruido, el Bias y el variance. EL Ensemble ayuda a minimizar estos factores. Estos métodos están diseñados para mejorar la estabilidad y precisión de los algoritmos de aprendizaje automático. Las combinaciones de múltiples clasificadores disminuyen la varianza, especialmente en el caso de clasificadores inestables, y pueden producir una clasificación más confiable que un solo clasificador.

1. En un Bagging y Boosting, tenemos N weak learners. Bagging and Boosting consigue N learners al generar datos adicionales en la etapa de entrenamiento. Se producen N nuevos conjuntos de datos de entrenamiento mediante un muestreo aleatorio con reemplazo del conjunto original. Al muestrear con reemplazo, algunas observaciones pueden repetirse en cada nuevo conjunto de datos de entrenamiento.

![BaggingBoosting_1](https://drive.google.com/uc?export=view&id=1kfwiVz97sbjxwbEQEjpt_IEIpDE98G1b)


2. En el caso del bagging, cualquier elemento tiene la misma probabilidad de aparecer en un nuevo conjunto de datos. Sin embargo, para Boosting las observaciones están ponderadas y, por lo tanto, algunas de ellas participarán en los nuevos conjuntos con más frecuencia:

![BaggingBoosting_2](https://drive.google.com/uc?export=view&id=1WJDzIDX-Nc-5t_qWDdxIQS7UH2SWGkkj)

3. En este punto, comenzamos a abordar la principal diferencia entre los dos métodos. Si bien la etapa de entrenamiento es paralela para Bagging (es decir, cada modelo se construye de forma independiente), Boosting construye al nuevo alumno de forma secuencial:

![BaggingBoosting_3](https://drive.google.com/uc?export=view&id=1sLd8mNEryln1om7SmKwbQULXv0EAyekT)

En los algoritmos boosting, cada clasificador se entrena con datos, teniendo en cuenta el resultado de los clasificadores anteriores. Después de cada paso de entrenamiento, los pesos se redistribuyen. Los datos mal clasificados aumentan su ponderación para enfatizar los casos más difíciles. De esta forma, los learners posteriores se centrarán en ellos durante su entrenamiento.

4. Para predecir la clase de nuevos datos, solo necesitamos aplicar los N learners a las nuevas observaciones. En Bagging, el resultado se obtiene promediando las respuestas de los N learners (o el voto de la mayoría). Sin embargo, Boosting asigna un segundo conjunto de ponderaciones, esta vez para los clasificadores N, con el fin de obtener un promedio ponderado de sus estimaciones. En la etapa de entrenamiento Boosting, el algoritmo asigna pesos a cada modelo resultante. A un learners con un buen resultado de clasificación en los datos de entrenamiento se le asignará un peso más alto que a uno deficiente. Por lo tanto, al evaluar a un nuevo learners, Boosting también debe realizar un seguimiento de los errores de los learners.

![BaggingBoosting_4](https://drive.google.com/uc?export=view&id=1eYDalb6ZBzvKhFWM89Ku-3BwrUzduUYq)




In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier # Usa muestreo boostrap

tree = DecisionTreeClassifier() # Learner
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, # Le pasamos los learners
                        random_state=1)

bag.fit(X, y)
visualize_classifier(bag, X, y)

En este ejemplo, hemos aleatorizado los datos ajustando cada estimador con un subconjunto aleatorio del 80% de los puntos de entrenamiento.
En la práctica, los árboles de decisión se aleatorizan de manera más efectiva inyectando algo de estocasticidad en la forma en que se eligen las divisiones: de esta manera, todos los datos contribuyen al ajuste cada vez, pero los resultados del ajuste aún tienen la aleatoriedad deseada.
Por ejemplo, al determinar qué característica dividir, el árbol aleatorio puede seleccionar entre las principales características.
Puede leer más detalles técnicos sobre estas estrategias de aleatorización en la [documentación de Scikit-Learn] (http://scikit-learn.org/stable/modules/ensemble.html#forest) y en las referencias.

En Scikit-Learn, este conjunto optimizado de árboles de decisión aleatorios se implementa en el estimador `` RandomForestClassifier '', que se encarga de toda la aleatorización automáticamente.
Todo lo que necesita hacer es seleccionar una serie de estimadores, y muy rápidamente (en paralelo, si lo desea) se ajustará al conjunto de árboles:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
visualize_classifier(model, X, y);

Vemos que al promediar más de 100 modelos perturbados aleatoriamente, terminamos con un modelo general que está mucho más cerca de nuestra intuición sobre cómo se debe dividir el espacio de parámetros.

## Random Forest para Regressión

En la sección anterior consideramos los random forest dentro del contexto de la clasificación.
Random forest también pueden funcionar en el caso de regresión (es decir, variables continuas en lugar de categóricas). El estimador a utilizar para esto es el `` RandomForestRegressor '', y la sintaxis es muy similar a la que vimos anteriormente.

Considere los siguientes datos, extraídos de la combinación de una oscilación rápida y lenta:

In [ ]:
rng = np.random.RandomState(42)
x = 10 * rng.rand(200)

def model(x, sigma=0.3):
    fast_oscillation = np.sin(5 * x)
    slow_oscillation = np.sin(0.5 * x)
    noise = sigma * rng.randn(len(x))

    return slow_oscillation + fast_oscillation + noise

y = model(x)
plt.errorbar(x, y, 0.3, fmt='o');

Usando el regresor de random forest, podemos encontrar la curva de mejor ajuste de la siguiente manera:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(200)
forest.fit(x[:, None], y)

xfit = np.linspace(0, 10, 1000)
yfit = forest.predict(xfit[:, None])
ytrue = model(xfit, sigma=0)

plt.errorbar(x, y, 0.3, fmt='o', alpha=0.5)
plt.plot(xfit, yfit, '-r'); # Este es el fit
plt.plot(xfit, ytrue, '-k', alpha=0.5);  # y este es el original

Aquí, el modelo real se muestra en la curva gris suave, mientras que el modelo de random forest se muestra en la curva roja irregular.
Como puede ver, el modelo de random forest no paramétrico es lo suficientemente flexible como para ajustarse a los datos de períodos múltiples, ¡sin que tengamos que especificar un modelo de períodos múltiples!

## <font color='green'>Actividad 2</font>

Clasificación de digitos. Esta actividad tiene por objetivo implementar un clasificador de digitos escritos a mano utilizando random forest.

1. Explore el data se y visualice datos. Puede utilizar el siguiente código para ver las imagenes. ¿Cuales son las dimensiones de la imágen?, ¿Entre que valores se mueve?

```
ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
```
2. Entrene un clasificador utilizando **RandomForestClassifier**. Explore algunos parámetros como por ejemplo.

* __max_depth__: La profundidad máxima del árbol. Si es None, los nodos se expanden hasta que todas las hojas sean puras o hasta que todas las hojas contengan menos de min_samples_split muestras.

* __n_estimators__: La cantidad de árboles a considerar.

* __max_features__: La cantidad de características que se deben considerar al buscar la mejor división:
<br>

3. Evalue los resultados a través de una matriz de confusión y ponga el resultado de una mariz en un heatmap para una mejor visualización.


In [ ]:
# Cargando y visualizando el dataset.
from sklearn.datasets import load_digits
digits = load_digits()
digits.keys()

Para recordarnos lo que estamos viendo, visualizaremos los primeros puntos de datos:

In [ ]:
# Tú codigo aquí

Podemos clasificar rápidamente los dígitos usando un random forest de la siguiente manera:

<font color='green'>Fin Actividad 2</font>

## Resumen del Método Random Forest:

Esta sección contiene una breve introducción al concepto de *ensamblados*, y en particular random forest, un conjunto de árboles de decisión aleatorios.
Random Forest es un método poderoso con varias ventajas:

- Tanto el entrenamiento como la predicción son muy rápidos, debido a la simplicidad de los árboles de decisión subyacentes. Además, ambas tareas se pueden paralelizar directamente, porque los árboles individuales son entidades completamente independientes.
- Los árboles múltiples permiten una clasificación probabilística: un voto mayoritario entre los estimadores da una estimación de la probabilidad (se accede en Scikit-Learn con el método ``predict_proba ()``).
- El modelo no paramétrico es extremadamente flexible y, por lo tanto, puede funcionar bien en tareas que no están ajustadas por otros estimadores.

Una desventaja principal de Random Forest es que los resultados no se pueden interpretar fácilmente: es decir, si desea sacar conclusiones sobre el *significado* del modelo de clasificación,  Random Forest pueden no ser la mejor opción.